In [6]:
import cv2
import numpy as np

img_path = 'D:\\Python\\1st DL\\Day002\\lena.png'

# 這裡設置 Saturation 的兩成
change_percentage = 0.2
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
# 轉換 hsv 往下調2成
# h -> 色調  s -> 飽和度  v -> 亮度
img_hsv_down_saturation = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_hsv_down_light = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 操作飽和度時需要轉換成百分比
img_hsv_down_saturation = img_hsv_down_saturation.astype("float32")
img_hsv_down_light = img_hsv_down_light.astype("float32")

# 轉成小數點並調降兩成飽和度
# 除以255後範圍變成
img_hsv_down_saturation[..., -1] = img_hsv_down_saturation[..., -1]/255 - change_percentage
img_hsv_down_light[..., 1] = img_hsv_down_light[..., 1]/255 - change_percentage

# 確保所有飽和度 channel 都介於0~1 之間
img_hsv_down_saturation[img_hsv_down_saturation[..., -1] < 0] = 0
img_hsv_down_light[img_hsv_down_light[..., 1] < 0] = 0

# 4捨5入，轉回0~255之間
img_hsv_down_saturation[..., -1] = np.around(img_hsv_down_saturation[..., -1]*255)
img_hsv_down_light[..., 1] = np.around(img_hsv_down_light[..., 1]*255)

# uint8
img_hsv_down_saturation = img_hsv_down_saturation.astype("uint8")
img_hsv_down_light = img_hsv_down_light.astype("uint8")


# 轉換 hsv 往上調2成
# h -> 色調 s -> 飽和度 v -> 亮度
img_hsv_up_saturation = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_hsv_up_light = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 操作飽和度時需要轉換成百分比
img_hsv_up_saturation = img_hsv_up_saturation.astype("float32")
img_hsv_up_light = img_hsv_up_light.astype("float32")

# 轉成小數點並調降兩成飽和度
# 除以255後範圍變成
img_hsv_up_saturation[..., -1] = img_hsv_up_saturation[..., -1]/255 + change_percentage
img_hsv_up_light[..., 1] = img_hsv_up_light[..., 1]/255 + change_percentage

# 確保所有飽和度 channel 都介於0~1 之間
img_hsv_up_saturation[img_hsv_up_saturation[..., -1] > 1] = 1
img_hsv_up_light[img_hsv_up_light[..., 1] > 1] = 1

# 4捨5入，轉回0~255之間
img_hsv_up_saturation[..., -1] = np.around(img_hsv_up_saturation[..., -1]*255)
img_hsv_up_light[..., 1] = np.around(img_hsv_up_light[..., 1]*255)

# uint8
img_hsv_up_saturation = img_hsv_up_saturation.astype("uint8")
img_hsv_up_light = img_hsv_up_light.astype("uint8")


# BGR
img_hsv_down_saturation = cv2.cvtColor(img_hsv_down_saturation, cv2.COLOR_HSV2BGR)
img_hsv_up_light = cv2.cvtColor(img_hsv_up_light, cv2.COLOR_HSV2BGR)

img_saturation_change = np.hstack((img, img_hsv_down_saturation, img_hsv_up_saturation))
img_light_change = np.hstack((img, img_hsv_down_light, img_hsv_up_light))

img_change = np.vstack((img_saturation_change, img_light_change))
cv2.imshow("img_change", img_change)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# 轉為灰階圖片
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 灰階圖片直方圖均衡
img_gray_equal = cv2.equalizeHist(img_gray)

img_gray_change = np.hstack((img_gray, img_gray_equal))

cv2.imshow("img_gray_change", img_gray_change)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# 對 RGB 圖直接處理

def hisEqulColor(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)
    channels = cv2.split(ycrcb)
    cv2.equalizeHist(channels[0], channels[1])
    cv2.merge(channels, ycrcb)
    cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR, image)
    return image

image = cv2.imread(img_path, cv2.IMREAD_COLOR)
img_rgb_equal = hisEqulColor(image)
img_rgb_change = np.hstack((img, img_rgb_equal))
cv2.imshow("img_rgb_change", img_rgb_change)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def convertScale(image, alpha, beta):
    for b in range(image.shape[0]):
        for g in range(image.shape[1]):
            for r in range(image.shape[2]):
                # 透過np.clib 限制範圍在0~255
                image[b,g,r] = np.clip(alpha * image[b,g,r] + beta, 0, 255)
    return image

image = cv2.imread(img_path, cv2.IMREAD_COLOR)
img_add_contrast = convertScale(cv2.imread(img_path, cv2.IMREAD_COLOR) , 2.0, 0)
img_add_contrast_cv = cv2.convertScaleAbs(image, alpha=2.0, beta=0)
img_add_lightness_cv = cv2.convertScaleAbs(image, alpha=1.0, beta=50)

img_change = np.hstack((image, img_add_contrast, img_add_contrast_cv, img_add_lightness_cv))
# img_change = np.hstack((image, img_add_contrast_cv, img_add_lightness_cv))

cv2.imshow("img_change", img_change)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def contrast_img(image, alpha, beta):  # 亮度就是每个像素所有通道都加上b
    rows, cols, channels = image.shape

    # 新建全零(黑色)图片数组:np.zeros(img1.shape, dtype=uint8)
    blank = np.zeros([rows, cols, channels], image.dtype)
    dst = cv2.addWeighted(image, alpha, blank, 1-alpha, beta)
    print(dst)
    change = np.hstack((image, dst))
    print(change)
    cv2.imshow('change', change)
    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()

contrast_img(img, 2, 30)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 ...

 [[150  66 198]
  [150  66 198]
  [146  84 214]
  ...
  [198 176 255]
  [182 166 255]
  [188 154 255]]

 [[144  74 194]
  [144  74 194]
  [154  94 222]
  ...
  [188 170 255]
  [192 172 255]
  [192 178 255]]

 [[144  74 194]
  [144  74 194]
  [154  94 222]
  ...
  [188 170 255]
  [192 172 255]
  [192 178 255]]]
[[[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [255 255 255]
  [250 255 255]
  [210 228 255]]

 ...

 [[ 60  18  84]
  [ 60  18  84]
  [ 58  27  92]
  ..

In [4]:
img_bgr = img

for i in range(3):
    img_bgr[..., i] = cv2.equalizeHist(img_bgr[..., i])
    
cv2.imshow('change', img_bgr)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()
